<span style="font-width:bold; font-size: 3rem; color:#333;">- Part 01: Feature Backfill for Air Quality Data</span>


## 🗒️ You have the following tasks
1. Choose an Air Quality Sensor
2. Update the country, city, and street information to point to YOUR chosen Air Quality Sensor
3. Download historical measures for your Air Quality Sensor as a CSV file
4. Update the path of the CSV file in this notebook to point to the one that you downloaded
5. Create an account on www.hopsworks.ai and get your HOPSWORKS_API_KEY
6. Run this notebook



In [59]:
import datetime
import requests
import pandas as pd
import hopsworks
import datetime
from pathlib import Path
import json
import re
import dotenv
import os

In [60]:
dotenv.load_dotenv()
project = hopsworks.login(engine="python")

2025-11-05 22:44:34,262 INFO: Closing external client and cleaning up certificates.
Connection closed.
2025-11-05 22:44:34,268 INFO: Initializing external client
2025-11-05 22:44:34,269 INFO: Base URL: https://c.app.hopsworks.ai:443
2025-11-05 22:44:34,989 WARNING: UserWarning: The installed hopsworks client version 4.4.2 may not be compatible with the connected Hopsworks backend version 4.2.2. 
To ensure compatibility please install the latest bug fix release matching the minor version of your backend (4.2) by running 'pip install hopsworks==4.2.*'



2025-11-05 22:44:35,957 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1279137


In [61]:



AQICN_API_KEY = os.getenv("AQICN_API_KEY")


print(f"Found AQICN_API_KEY: {AQICN_API_KEY}")

secrets = hopsworks.get_secrets_api()
# Replace any existing secret with the new value
secret = secrets.get_secret("AQICN_API_KEY")
if secret is not None:
    secret.delete()
    print("Replacing existing AQICN_API_KEY")

secrets.create_secret("AQICN_API_KEY", AQICN_API_KEY)

Found AQICN_API_KEY: 708f4173a90a3315ba6464933d6964b5bc6fc765
Replacing existing AQICN_API_KEY
Secret created successfully, explore it at https://c.app.hopsworks.ai:443/account/secrets


Secret('AQICN_API_KEY', 'PRIVATE')

In [62]:
sensor_id_cental = 1666
sensor_id_east = 1664
sensor_id_west = 1665
sensor_id_north = 1662
sensor_id_south = 1663

sensor_id = sensor_id_cental

url = f"https://api.waqi.info/feed/@{sensor_id}/?token={AQICN_API_KEY}"

response = requests.get(url)
if response.status_code == 200:
    data = response.json()
else:
    print("Failed to retrieve data. Status Code:", response.status_code)
    raise requests.exceptions.RequestException(response.status_code)

if data['status'] != 'ok':
    raise Exception(f"Error: {data['status']}")

aqi_data = data['data']
aq_today_df = pd.DataFrame()
aq_today_df['pm25'] = [aqi_data['iaqi'].get('pm25', {}).get('v', None)]
aq_today_df['pm25'] = aq_today_df['pm25'].astype('float32')

aq_today_df['sensor_id'] = sensor_id
aq_today_df['city'] = "Singapore"



In [63]:
aq_today_df.head()

,pm25,sensor_id,city
0,85.0,1666,Singapore


In [64]:
csv_file = "../data/central,-singapore-air-quality.csv"


df = pd.read_csv(csv_file,  parse_dates=['date'], skipinitialspace=True)

df

,date,pm25,pm10,o3,no2,so2,co,psi
0,2025-11-01,63.0,27.0,13.0,NaN,1.0,3.0,NaN
1,2025-11-02,47.0,24.0,10.0,NaN,2.0,3.0,NaN
2,2025-11-03,50.0,30.0,23.0,NaN,3.0,3.0,NaN
3,2025-11-04,71.0,32.0,15.0,NaN,2.0,3.0,NaN
4,2025-11-05,68.0,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...
4313,2015-06-29,NaN,23.0,21.0,13.0,3.0,5.0,60.0
4314,2015-06-30,NaN,27.0,10.0,20.0,3.0,5.0,58.0
4315,2014-12-31,NaN,23.0,34.0,16.0,3.0,1.0,NaN
4316,2014-07-20,NaN,17.0,14.0,7.0,6.0,3.0,47.0


In [65]:
df_aq = df[['date', 'pm25']]
df_aq['pm25'] = df_aq['pm25'].astype('float32')
df_aq['city'] = "Singapore"
df_aq['sensor_id'] = sensor_id

2025-11-05 22:44:38,832 WARNING: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

2025-11-05 22:44:38,834 WARNING: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

2025-11-05 22:44:38,836 WARNING: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [66]:
# Cast the pm25 column to be a float32 data type
df_aq.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4318 entries, 0 to 4317
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype         
---  ------     --------------  -----         
 0   date       4318 non-null   datetime64[ns]
 1   pm25       4285 non-null   float32       
 2   city       4318 non-null   object        
 3   sensor_id  4318 non-null   int64         
dtypes: datetime64[ns](1), float32(1), int64(1), object(1)
memory usage: 118.2+ KB


In [67]:
df_aq=df_aq[df_aq["date"] >= "2016-01-01"]

In [68]:
# Filter where pm25 is null and sort descending by date
null_dates = (
    df_aq.loc[df_aq['pm25'].isna(), 'date']
    .sort_values(ascending=False)
)

null_dates


4285   2024-12-31
4286   2020-03-31
4287   2019-12-31
4288   2018-12-31
4289   2018-03-31
4290   2017-09-10
4291   2016-01-03
Name: date, dtype: datetime64[ns]

In [69]:
df_aq.dropna(inplace=True)
df_aq

,date,pm25,city,sensor_id
0,2025-11-01,63.0,Singapore,1666
1,2025-11-02,47.0,Singapore,1666
2,2025-11-03,50.0,Singapore,1666
3,2025-11-04,71.0,Singapore,1666
4,2025-11-05,68.0,Singapore,1666
...,...,...,...,...
3577,2016-03-28,65.0,Singapore,1666
3578,2016-03-29,104.0,Singapore,1666
3579,2016-03-30,94.0,Singapore,1666
3580,2016-03-31,104.0,Singapore,1666


In [70]:
df_aq.info()

<class 'pandas.core.frame.DataFrame'>
Index: 3582 entries, 0 to 3673
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype         
---  ------     --------------  -----         
 0   date       3582 non-null   datetime64[ns]
 1   pm25       3582 non-null   float32       
 2   city       3582 non-null   object        
 3   sensor_id  3582 non-null   int64         
dtypes: datetime64[ns](1), float32(1), int64(1), object(1)
memory usage: 125.9+ KB


## <span style='color:#ff5f27'> 🌦 Loading Weather Data from [Open Meteo](https://open-meteo.com/en/docs)

In [71]:
earliest_aq_date = pd.Series.min(df_aq['date'])
earliest_aq_date = earliest_aq_date.strftime('%Y-%m-%d')
earliest_aq_date

'2016-01-01'

In [72]:

url = "https://archive-api.open-meteo.com/v1/archive"

latitude = 1.3667
longitude = 103.8

params = {
    "latitude": latitude,
    "longitude": longitude,
    "start_date": earliest_aq_date,
    "format": "json",
    "end_date": datetime.datetime.now().strftime("%Y-%m-%d"),
    "daily": ["temperature_2m_mean", "precipitation_sum", "wind_speed_10m_max", "wind_direction_10m_dominant"]

}


response = requests.get(url, params=params)


if response.status_code == 200:
    # Extract the JSON content from the response
    data = response.json()
else:
    print("Failed to retrieve data. Status Code:", response.status_code)
    raise requests.exceptions.RequestException(response.status_code)




In [73]:

d = data["daily"]

df = pd.DataFrame({
    "date": pd.to_datetime(d["time"]),
    "temperature_2m_mean": d["temperature_2m_mean"],
    "precipitation_sum": d["precipitation_sum"],
    "wind_speed_10m_max": d["wind_speed_10m_max"],
    "wind_direction_10m_dominant": d["wind_direction_10m_dominant"],
})

df["city"] = "Singapore"


In [74]:
# Check how many nulls exist per column
print(df.isna().sum())

# Show only rows where ANY column has a null value
df[df.isna().any(axis=1)]

date                           0
temperature_2m_mean            0
precipitation_sum              0
wind_speed_10m_max             0
wind_direction_10m_dominant    0
city                           0
dtype: int64


,date,temperature_2m_mean,precipitation_sum,wind_speed_10m_max,wind_direction_10m_dominant,city


In [75]:
weather_df = df

In [76]:
weather_df.info()



<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3597 entries, 0 to 3596
Data columns (total 6 columns):
 #   Column                       Non-Null Count  Dtype         
---  ------                       --------------  -----         
 0   date                         3597 non-null   datetime64[ns]
 1   temperature_2m_mean          3597 non-null   float64       
 2   precipitation_sum            3597 non-null   float64       
 3   wind_speed_10m_max           3597 non-null   float64       
 4   wind_direction_10m_dominant  3597 non-null   int64         
 5   city                         3597 non-null   object        
dtypes: datetime64[ns](1), float64(3), int64(1), object(1)
memory usage: 168.7+ KB


## <span style='color:#ff5f27'> 🌍 STEP 10: Define Data Validation Rules </span>

We will validate the air quality measurements (`pm25` values) before we write them to Hopsworks.

We define a data validation rule (an expectation in Great Expectations) that ensures that `pm25` values are not negative or above the max value available by the sensor.

We will attach this expectation to the air quality feature group, so that we validate the `pm25` data every time we write a DataFrame to the feature group. We want to prevent garbage-in, garbage-out.

In [77]:
import great_expectations as ge


aq_expectation_suite = ge.core.ExpectationSuite(
    expectation_suite_name="aq_expectation_suite"
)

aq_expectation_suite.add_expectation(
    ge.core.ExpectationConfiguration(
        expectation_type="expect_column_min_to_be_between",
        kwargs={
            "column":"pm25",
            "min_value":-0.1,
            "max_value":500.0,
            "strict_min":True
        }
    )
)

{"expectation_type": "expect_column_min_to_be_between", "kwargs": {"column": "pm25", "min_value": -0.1, "max_value": 500.0, "strict_min": true}, "meta": {}}

## Expectations for Weather Data
Here, we define an expectation for 2 columns in our weather DataFrame - `precipitation_sum` and `wind_speed_10m_max`, where we expect both values to be greater than zero, but less than 1000.

In [78]:
import great_expectations as ge
weather_expectation_suite = ge.core.ExpectationSuite(
    expectation_suite_name="weather_expectation_suite"
)

def expect_greater_than_zero(col):
    weather_expectation_suite.add_expectation(
        ge.core.ExpectationConfiguration(
            expectation_type="expect_column_min_to_be_between",
            kwargs={
                "column":col,
                "min_value":-0.1,
                "max_value":1000.0,
                "strict_min":True
            }
        )
    )
expect_greater_than_zero("precipitation_sum")
expect_greater_than_zero("wind_speed_10m_max")

---

### <span style="color:#ff5f27;"> 🔮 STEP 11: Connect to Hopsworks and save the sensor country, city, street names as a secret</span>

In [79]:
fs = project.get_feature_store() 

#### Save country, city, street names as a secret

These will be downloaded from Hopsworks later in the (1) daily feature pipeline and (2) the daily batch inference pipeline

In [80]:
dict_obj = {
    "country": "Singapore",
    "city": "Singapore",
    "street": "Central",
    "sensor_id": sensor_id,
    "latitude": latitude,
    "longitude": longitude
}

# Convert the dictionary to a JSON string
str_dict = json.dumps(dict_obj)

# Replace any existing secret with the new value
secret = secrets.get_secret("SENSOR_LOCATION_JSON")
if secret is not None:
    secret.delete()
    print("Replacing existing SENSOR_LOCATION_JSON")

secrets.create_secret("SENSOR_LOCATION_JSON", str_dict)

Replacing existing SENSOR_LOCATION_JSON
Secret created successfully, explore it at https://c.app.hopsworks.ai:443/account/secrets


Secret('SENSOR_LOCATION_JSON', 'PRIVATE')

### <span style="color:#ff5f27;"> 🔮 STEP 12: Create the Feature Groups and insert the DataFrames in them </span>

### <span style='color:#ff5f27'> 🌫 Air Quality Data
    
 1. Provide a name, description, and version for the feature group.
 2. Define the `primary_key`: we have to select which columns uniquely identify each row in the DataFrame - by providing them as the `primary_key`. Here, each air quality sensor measurement is uniquely identified by `country`, `street`, and  `date`.
 3. Define the `event_time`: We also define which column stores the timestamp or date for the row - `date`.
 4. Attach any `expectation_suite` containing data validation rules

In [81]:
air_quality_fg = fs.get_or_create_feature_group(
    name='air_quality',
    description='Air Quality characteristics of each day',
    version=1,
    primary_key=['city', 'sensor_id'],
    event_time="date",
    expectation_suite=aq_expectation_suite
)

#### Insert the DataFrame into the Feature Group

In [82]:
air_quality_fg.insert(df_aq)

Feature Group created successfully, explore it at 
https://c.app.hopsworks.ai:443/p/1279137/fs/1265747/fg/1596048
2025-11-05 22:44:42,729 INFO: 	1 expectation(s) included in expectation_suite.
Validation succeeded.
Validation Report saved successfully, explore a summary at https://c.app.hopsworks.ai:443/p/1279137/fs/1265747/fg/1596048


Uploading Dataframe: 100.00% |██████████| Rows 3582/3582 | Elapsed Time: 00:02 | Remaining Time: 00:00


Launching job: air_quality_1_offline_fg_materialization
Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai:443/p/1279137/jobs/named/air_quality_1_offline_fg_materialization/executions


(Job('air_quality_1_offline_fg_materialization', 'SPARK'),
 {
   "success": true,
   "results": [
     {
       "success": true,
       "expectation_config": {
         "expectation_type": "expect_column_min_to_be_between",
         "kwargs": {
           "column": "pm25",
           "min_value": -0.1,
           "max_value": 500.0,
           "strict_min": true
         },
         "meta": {
           "expectationId": 733244
         }
       },
       "result": {
         "observed_value": 12.999999046325684,
         "element_count": 3582,
         "missing_count": null,
         "missing_percent": null
       },
       "meta": {
         "ingestionResult": "INGESTED",
         "validationTime": "2025-11-05T09:44:42.000727Z"
       },
       "exception_info": {
         "raised_exception": false,
         "exception_message": null,
         "exception_traceback": null
       }
     }
   ],
   "evaluation_parameters": {},
   "statistics": {
     "evaluated_expectations": 1,
     "su

#### Enter a description for each feature in the Feature Group

In [84]:
air_quality_fg.update_feature_description("date", "Date of measurement of air quality")
air_quality_fg.update_feature_description("city", "City where the air quality was measured")
air_quality_fg.update_feature_description("pm25", "Particles less than 2.5 micrometers in diameter (fine particles) pose health risk")
air_quality_fg.update_feature_description("sensor_id", "Sensor ID of the air quality measurement")

### <span style='color:#ff5f27'> 🌦 Weather Data
    
 1. Provide a name, description, and version for the feature group.
 2. Define the `primary_key`: we have to select which columns uniquely identify each row in the DataFrame - by providing them as the `primary_key`. Here, each weather measurement is uniquely identified by `city` and  `date`.
 3. Define the `event_time`: We also define which column stores the timestamp or date for the row - `date`.
 4. Attach any `expectation_suite` containing data validation rules

In [ ]:
# Get or create feature group 
weather_fg = fs.get_or_create_feature_group(
    name='weather',
    description='Weather characteristics of each day',
    version=1,
    primary_key=['city'],
    event_time="date",
    expectation_suite=weather_expectation_suite
) 

: 

: 

: 

: 

: 

: 

#### Insert the DataFrame into the Feature Group

In [ ]:
# Insert data
weather_fg.insert(weather_df, wait=True)

: 

: 

: 

: 

: 

: 

#### Enter a description for each feature in the Feature Group

In [ ]:
weather_fg.update_feature_description("date", "Date of measurement of weather")
weather_fg.update_feature_description("city", "City where weather is measured/forecast for")
weather_fg.update_feature_description("temperature_2m_mean", "Temperature in Celsius")
weather_fg.update_feature_description("precipitation_sum", "Precipitation (rain/snow) in mm")
weather_fg.update_feature_description("wind_speed_10m_max", "Wind speed at 10m abouve ground")
weather_fg.update_feature_description("wind_direction_10m_dominant", "Dominant Wind direction over the dayd")

: 

: 

: 

: 

: 

: 

## <span style="color:#ff5f27;">⏭️ **Next:** Part 02: Daily Feature Pipeline 
 </span> 


## <span style="color:#ff5f27;">⏭️ **Exercises:** 
 </span> 

Extra Homework:

  * Try adding a new feature based on a rolling window of 3 days for 'pm25'
      * This is not easy, as forecasting more than 1 day in the future, you won't have the previous 3 days of pm25 measurements.
      * df.set_index("date").rolling(3).mean() is only the start....
  * Parameterize the notebook, so that you can provide the `country`/`street`/`city`/`url`/`csv_file` as parameters. 
      * Hint: this will also require making the secret name (`SENSOR_LOCATION_JSON`), e.g., add the street name as part of the secret name. Then you have to pass that secret name as a parameter when running the operational feature pipeline and batch inference pipelines.
      * After you have done this, collect the street/city/url/csv files for all the sensors in your city or region and you make dashboards for all of the air quality sensors in your city/region. You could even then add a dashboard for your city/region, as done [here for Poland](https://github.com/erno98/ID2223).

Improve this AI System
  * As of mid 2024, there is no API call available to download historical data from the AQIN website. You could improve this system by writing a PR to download the CSV file using Python Selenium and the URL for the sensor.


---